In [ ]:
import pandas as pd
import geopandas as gpd
import sqlalchemy as sa
import pyodbc
import tkinter as tk

In [ ]:
# Define custom colour palette

humannature = {'GC Dark Green': '#294238',
               'GC Light Green': '#b2d235',
               'GC Mid Green': '#50b748',
               'GC Orange': '#f57821',
               'GC Light Grey': '#e6ebe3'
               }

In [ ]:
# SQL server details 
server = 'dblistener1'
database = 'information_centre'
driver = 'ODBC Driver 17 for SQL Server' # Create SQLAlchemy engine and install necessary drivers
connection_string = f'mssql+pyodbc://{server}/{database}?driver={driver}'
engine = create_engine(connection_string)

# SQL query to select dataset from SSMS tables and save into allvisits dataframe
query = """"
USE information_centre;

SELECT
    a.intContractorID,
    a.strName AS Contractor,
    e.BusinessUnitID,
    e.Name AS BusinessUnit,
    UPPER(a.strPostcode) AS Postcode,
    g.Latitude,
    g.Longitude,
    a.InternalContractor
FROM
    tblContractor a
    LEFT JOIN Contractor_Business_Unit d
        ON a.intContractorID = d.ContractorID
    LEFT JOIN Business_Unit e
        ON d.BusinessUnitID = e.BusinessUnitID
    LEFT JOIN Business_Unit_Master_Status f
        ON d.StatusID = f.StatusID
    LEFT JOIN dbs_PostCode.dbo.tblPostcodes_New g
        ON REPLACE(a.strPostcode, ' ', '') = g.PostcodeNoSpaces COLLATE Latin1_General_CI_AS
WHERE
    ISNULL(a.bDisabled, 0) = 0
    AND f.StatusID IN (60, 70, 80)
    AND ISNULL(a.IsTest, 0) = 0
    AND e.BusinessUnitID != 37;
"""
with engine.connect() as connection:
    allvisits = pd.read_sql(query, connection)
    connection.close()

SyntaxError: invalid syntax (2493837984.py, line 37)